<a href="https://colab.research.google.com/github/Viny2030/NLP/blob/main/100_times_faster_nlp_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 100 Times Faster Natural Language Processing in Python

This iPython notebook contains the examples detailed in my post [🚀 100 Times Faster Natural Language Processing in Python](https://medium.com/huggingface/100-times-faster-natural-language-processing-in-python-ee32033bdced).

To run the notebook, you will first need to:
- [install Cython](http://cython.readthedocs.io/en/latest/src/quickstart/install.html), e.g. ```pip install cython```
- [install spaCy](https://spacy.io/usage/), e.g. ```pip install spacy```
- [download a language model for spaCy](https://spacy.io/usage/models), e.g. ```python -m spacy download en```

Cython then has to be activated in the notebook as follows:

Este cuaderno de Ipython contiene los ejemplos detallados en mi publicación 🚀 🚀 🚀 🚀 100 veces el procesamiento de lenguaje natural más rápido en Python.

Para ejecutar el cuaderno, primero deberá:

Instale Cython, p. PIP Instale Cython
Instale Spacy, p. PIP install Spacy
Descargue un modelo de idioma para Spacy, p. Ej. Python -M Spacy Descargar en
Cython debe activarse en el cuaderno de la siguiente manera:

In [1]:
%load_ext Cython

# Fast loops in Python with a bit of Cython

![Rectangles](https://cdn-images-1.medium.com/max/800/0*RA89oQ-0j3Rscipw.jpg "Rectangles")

In this simple example we have a large set of rectangles that we store as a list of Python objects, e.g. instances of a Rectangle class. The main job of our module is to iterate over this list in order to count how many rectangles have an area larger than a specific threshold.

Our Python module is quite simple and looks like this (see also here: https://gist.github.com/thomwolf/0709b5a72cf3620cd00d94791213d38e):

En este simple ejemplo, tenemos un gran conjunto de rectángulos que almacenamos como una lista de objetos de Python, p. instancias de una clase rectangular. El trabajo principal de nuestro módulo es iterar sobre esta lista para contar cuántos rectángulos tienen un área más grande que un umbral específico.

Nuestro módulo Python es bastante simple y se ve así (ver también aquí: https://gist.github.com/thomwolf/0709b5a72cf3620cd00d94791213d38e):


In [2]:
from random import random

class Rectangle:
    def __init__(self, w, h):
        self.w = w
        self.h = h
    def area(self):
        return self.w * self.h

def check_rectangles_py(rectangles, threshold):
    n_out = 0
    for rectangle in rectangles:
        if rectangle.area() > threshold:
            n_out += 1
    return n_out

def main_rectangles_slow():
    n_rectangles = 10000000
    rectangles = list(Rectangle(random(), random()) for i in range(n_rectangles))
    n_out = check_rectangles_py(rectangles, threshold=0.25)
    print(n_out)

In [3]:
%%time
# Let's run it:
main_rectangles_slow()

4034324
CPU times: user 14 s, sys: 1.8 s, total: 15.8 s
Wall time: 15.8 s


The ```check_rectangles``` function which loops over a large number of Python objects is our bottleneck!

Let's write it in Cython.

We indicate the cell is a Cython cell by using the ```%%cython``` magic command. We the cell is run, the cython code will be written in a temporary file, compiled and reimported in the iPython space. The Cython code thus have to be somehow self contained.

¡La función check_rectangles que gira sobre una gran cantidad de objetos de pitón es nuestro cuello de botella!

Vamos a escribirlo en Cython.

Indicamos que la célula es una célula Cython usando el comando Magic %% Cython. Se ejecuta la celda, el código Cython se escribirá en un archivo temporal, compilado y reimportado en el espacio de Ipython. El Código Cython tiene que ser de alguna manera auto contenido.

In [4]:
%%cython
from cymem.cymem cimport Pool
from random import random

cdef struct Rectangle:
    float w
    float h

cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    cdef int n_out = 0
    # C arrays contain no size information => we need to state it explicitly
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out

def main_rectangles_fast():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.25
    cdef Pool mem = Pool()
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

In [5]:
%%time
main_rectangles_fast()

4035936
CPU times: user 585 ms, sys: 49 ms, total: 635 ms
Wall time: 632 ms


In this simple case we are about 20 times faster in Cython.

The ratio of improvement depends a lot on the specific syntax of the Python program.

While the speed in Cython is rather predictible once your code make only use of C level objects (it is usually directly the fastest possible speed), the speed of Python can vary a lot depending on how your program is written and how much overhead the interpreter will add.

En este simple caso somos unas 20 veces más rápidas en Cython.

La relación de mejora depende mucho de la sintaxis específica del programa Python.

Si bien la velocidad en Cython es bastante predicible una vez que su código solo hace uso de los objetos de nivel C (generalmente es directamente la velocidad más rápida posible), la velocidad de Python puede variar mucho dependiendo de cómo esté escrito su programa y cuánta sobrecarga se intenta el intérprete agregará.


How can you be sure you Cython program makes only use of C level structures?

Use the ```-a``` or ```--annotate``` flag in the ```%%cython``` magic command to display a code analysis with the line accessing and using Python objects highlighted in yellow.

Here is how our the code analysis of previous program looks:

¿Cómo puede estar seguro de que el programa Cython hace solo el uso de estructuras de nivel C?

Use el indicador -a o -anotate en el comando %% mágico de Cython para mostrar un análisis de código con la línea de acceso y usar objetos de pitón resaltados en amarillo.

Así es como se ve nuestro análisis de código del programa anterior:

In [6]:
%%cython -a
from cymem.cymem cimport Pool
from random import random

cdef struct Rectangle:
    float w
    float h

cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    cdef int n_out = 0
    # C arrays contain no size information => we need to state it explicitly
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out

cpdef main_rectangles_fast():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.25
    cdef Pool mem = Pool()
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

The important element here is that lines 11 to 13 are not highlighted which means they will be running at the fastest possible speed.

It's ok to have yellow lines in the ```main_rectangle_fast``` function as this function will only be called once when we execute our program anyway. The yellow lines 22 and 23 are initialization lines that we could avoid by using a C level random function like `stdlib rand()` but we didn't want to clutter this example.

Now here is an example of the previous cython program not optimized (with Python objects in the loop):

El elemento importante aquí es que las líneas 11 a 13 no se resaltan, lo que significa que se ejecutarán a la velocidad más rápida posible.

Está bien tener líneas amarillas en la función main_rectangle_fast, ya que esta función solo se llamará una vez cuando ejecutemos nuestro programa de todos modos. Las líneas amarillas 22 y 23 son líneas de inicialización que podríamos evitar utilizando una función aleatoria de nivel C como stdlib rand () pero no queríamos desordenar este ejemplo.

Ahora aquí hay un ejemplo del programa Cython anterior no optimizado (con objetos de Python en el bucle):


In [7]:
%%cython -a
from cymem.cymem cimport Pool
from random import random

cdef struct Rectangle:
    float w
    float h

cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    # ========== MODIFICATION ===========
    # We changed the following line from `cdef int n_out = 0` to
    n_out = 0
    # n_out is not defined as an `int` anymore and is now thus a regular Python object
    # ===================================
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out

cpdef main_rectangles_not_so_fast():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.25
    cdef Pool mem = Pool()
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

We can see that line 16 in the loop of `check_rectangles_cy` is highlighted, indicating that the Cython compiler had to add some Python API overhead.


Podemos ver que la línea 16 en el bucle de check_rectangles_cy se resalta, lo que indica que el compilador de Cython tuvo que agregar un poco de sobrecarga de la API de Python.

# 💫 Using Cython with spaCy to speed up NLP

Our blog post go in some details about the way spaCy can help you speed up your code by using Cython for NLP.

Here is a short summary of the post:
- the official Cython documentation advises against the use of C strings: `Generally speaking: unless you know what you are doing, avoid using C strings where possible and use Python string objects instead.`
- spaCy let us overcome this problem by:
    - converting all strings to 64-bit hashes using a look up between Python unicode strings and 64-bit hashes called the `StringStore`
    - giving us access to fully populated C level structures of the document and vocabulary called `TokenC` and `LexemeC`

The `StringStore` object is accessible from everywhere in spaCy and every object (see on the left), for example as `nlp.vocab.strings`, `doc.vocab.strings` or `span.doc.vocab.string`:

Nuestra publicación de blog va en algunos detalles sobre la forma en que Spacy puede ayudarlo a acelerar su código usando Cython para PNL.

Aquí hay un breve resumen de la publicación:

La documentación oficial de Cython aconseja contra el uso de cadenas C: Generalmente en términos generales: a menos que sepa lo que está haciendo, evite usar cadenas C siempre que sea posible y use objetos de cadena Python.
SPACY superamos este problema por:
Convertir todas las cadenas a hashes de 64 bits utilizando una mirada hacia arriba entre las cadenas unicode de Python y los hashes de 64 bits llamados Stringstore
dándonos acceso a estructuras de nivel C totalmente pobladas del documento y vocabulario llamados Tokenc y Lexemec
Se puede acceder al objeto StringStore desde todas partes en Spacy y cada objeto (ver a la izquierda), por ejemplo, como NLP.Vocab.Strings, doc.vocab.strings o span.doc.vocab.string:

![spaCy's internals](https://cdn-images-1.medium.com/max/600/1*nxvhI7mEc9A75PwMH-PSBg.png "spaCy's internals")

Here is now a simple example of NLP processing in Cython.

First let's build a list of big documents and parse them using spaCy (this takes a few minutes):

Aquí hay un ejemplo simple de procesamiento de PNL en Cython.

Primero construamos una lista de grandes documentos y los analicemos usando Spacy (esto lleva unos minutos):

In [9]:
import urllib.request
import spacy

# Download the spaCy model if you haven't already
!python -m spacy download en_core_web_sm

# Build a dataset of 10 parsed document extracted from the Wikitext-2 dataset
with urllib.request.urlopen('https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/valid.txt') as response:
   text = response.read()
# Load the spaCy model using its full name
nlp = spacy.load('en_core_web_sm')
doc_list = list(nlp(text[:800000].decode('utf8')) for i in range(10))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


We have about 1.7 million tokens ("words") in our dataset:

Tenemos alrededor de 1.7 millones de tokens ("palabras") en nuestro conjunto de datos:

In [ ]:
sum(len(doc) for doc in doc_list)

1716200

We want to perform some NLP task on this dataset.

For example, we would like to count the number of times the word "run" is used as a noun in the dataset (i.e. tagged with a "NN" Part-Of-Speech tag).

A Python loop to do that is short and straightforward:

Queremos realizar alguna tarea NLP en este conjunto de datos.

Por ejemplo, nos gustaría contar el número de veces que la palabra "ejecutar" se usa como un sustantivo en el conjunto de datos (es decir, etiquetado con una etiqueta "nn" de voz).

Un bucle de pitón para hacer es corto y sencillo:

In [11]:
def slow_loop(doc_list, word, tag):
    n_out = 0
    for doc in doc_list:
        for tok in doc:
            if tok.lower_ == word and tok.tag_ == tag:
                n_out += 1
    return n_out

def main_nlp_slow(doc_list):
    n_out = slow_loop(doc_list, 'run', 'NN')
    print(n_out)

In [12]:
%%time
# But it's also quite slow
main_nlp_slow(doc_list)

90
CPU times: user 729 ms, sys: 1.04 ms, total: 730 ms
Wall time: 729 ms


On my laptop this code takes about 1.4 second to get the answer.

Let's try to speed this up with spaCy and a bit of Cython.

First, we have to think about the data structure. We will need a C level array for the dataset, with pointers to each document's TokenC array. We'll also need to convert the strings we use for testing to 64-bit hashes: "run" and "NN". When all the data required for our processing is in C level objects, we can then iterate at full C speed over the dataset.

Here is how this example can be written in Cython with spaCy:

En mi computadora portátil, este código tarda aproximadamente 1.4 segundos en obtener la respuesta.

Intentemos acelerar esto con Spacy y un poco de Cython.

Primero, tenemos que pensar en la estructura de datos. Necesitaremos una matriz de nivel C para el conjunto de datos, con punteros para la matriz de tokenc de cada documento. También tendremos que convertir las cadenas que usamos para probar a hashes de 64 bits: "Ejecutar" y "NN". Cuando todos los datos requeridos para nuestro procesamiento están en objetos de nivel C, podemos iterar a una velocidad C completa a través del conjunto de datos.

Así es como este ejemplo se puede escribir en Cython con Spacy:
Enviar comentarios


In [13]:
%%cython -+
import numpy # Sometime we have a fail to import numpy compilation error if we don't import numpy
from cymem.cymem cimport Pool
from spacy.tokens.doc cimport Doc
from spacy.typedefs cimport hash_t
from spacy.structs cimport TokenC

cdef struct DocElement:
    TokenC* c
    int length

cdef int fast_loop(DocElement* docs, int n_docs, hash_t word, hash_t tag):
    cdef int n_out = 0
    for doc in docs[:n_docs]:
        for c in doc.c[:doc.length]:
            if c.lex.lower == word and c.tag == tag:
                n_out += 1
    return n_out

cpdef main_nlp_fast(doc_list):
    cdef int i, n_out, n_docs = len(doc_list)
    cdef Pool mem = Pool()
    cdef DocElement* docs = <DocElement*>mem.alloc(n_docs, sizeof(DocElement))
    cdef Doc doc
    for i, doc in enumerate(doc_list): # Populate our database structure
        docs[i].c = doc.c
        docs[i].length = (<Doc>doc).length
    word_hash = doc.vocab.strings.add('run')
    tag_hash = doc.vocab.strings.add('NN')
    n_out = fast_loop(docs, n_docs, word_hash, tag_hash)
    print(n_out)

Content of stderr:
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /root/.cache/ipython/cython/_cython_magic_de22b82129f3fcb5dcf74019e21a856c9ffc7484.cpp:1262:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

In [14]:
%%time
main_nlp_fast(doc_list)

90
CPU times: user 20.3 ms, sys: 0 ns, total: 20.3 ms
Wall time: 24.4 ms


The code is a bit longer because we have to declare and populate the C structures in `main_nlp_fast` before calling our Cython function.

But it is also a lot faster! In my Jupyter notebook, this cython code takes about 21 milliseconds to run on my laptop which is about **60 times faster** than our previous pure Python loop.

El código es un poco más largo porque tenemos que declarar y llenar las estructuras C en main_nlp_fast antes de llamar a nuestra función Cython.

¡Pero también es mucho más rápido! En mi cuaderno de Jupyter, este código Cython tarda alrededor de 21 milisegundos en funcionar en mi computadora portátil, que es aproximadamente 60 veces más rápido que nuestro bucle de pitón puro anterior.

The absolute speed is also impressive for a module written in an interactive Jupyter Notebook and which can interface natively with other Python modules and functions: scanning ~1,7 million words in 18ms means we are processing **a whopping 80 millions words per seconds**.


La velocidad absoluta también es impresionante para un módulo escrito en un cuaderno interactivo de Jupyter y que puede interactuar de forma nativa con otros módulos y funciones de Python: escanear ~ 1,7 millones de palabras en 18ms significa que estamos procesando 80 millones de palabras por segundo.